**目的:** 
  分析不同群組之間是否有*表現*上的不同

**範圍定義:**

  時間: 2020年的購買保單行為，但部分特徵是2017~2020的累計(e.g.曾買過的險種)

  群組: 
  
    1. 2020所有客戶                                                 → 比較基礎
    2. 2020使用主要通路的客戶                                        → 比較不同通路(不細分來源)
    3. 2019使用通路ₐ(以下稱*通路ₐ*) 2020使用通路ᵦ(以下稱*通路ᵦ*)的客戶 → 比較相同來源，不同通路
    
  群組詳細說明:
    
    以車險為例:
    1. 2020買過{車險}的客戶的所有保單
    2. 2020用{通路r}買過{車險}的客戶，在2020的{所有保單} (不分險別、不論通路)
    3. 2019用{通路ₐ}買過{車險}，而且2020使用{通路ᵦ}買過{車險}的客戶，在2020年的{所有保單} (不分險別、不論通路)

### notes:
2022/09/06 會議: 

* 特徵加上:
-人數
-出險人數佔比
-車數PR75

2022/09/07 討論: 也許只有在算錢的時候才只排除強制險? (因為就使用通路的角度，強制險的客戶就是有用到阿)


In [2]:
# %% import & read data
from src import group_feat as gf
import pandas as pd
import pickle

In [3]:
e = gf.env()
data = e.used_data
STATS_YEAR = 2020
MAIN_ROUTES = ["CA", "KA", "JB", "BA", "KB"]
data["iroute_"] = data["iroute"]
data.loc[~data["iroute_"].isin(MAIN_ROUTES), "iroute_"] = "others"


In [ ]:
cust_r = data.query("year==@STATS_YEAR & cate=='CA_PS' & iroute=='CA' ")["id"].unique() #2020有用過通路_r買車險的客戶

In [ ]:
cust_2020 = data.query(" year == @STATS_YEAR & cate=='CA_PS' ")["id"].unique() # 2020年的客戶

In [ ]:
# Q: 為什麼客戶一定有買過車險，但by通路之後卻沒有了呢?
# for r in MAIN_ROUTES:
#     df = data.query(" year==@STATS_YEAR & iroute==@r & id in @cust_2020")
#     print(df.query(" cate=='CA_PS '")["id"].nunique() / df["id"].nunique())
# A: 因為在這個通路篩選下的訂單中，他沒有用這個通路買過車險
# Q: 但是如果只篩選"在這通路底下用過車險的客戶"，又會少掉一大堆客戶。
# 所以問題的定義會被改成: 特定客戶用特定通路買過某險別的比例。(客戶可能買過這個險別，但不是在這個通路底下)
# 但我想知道的是: 使用過此通路的特定客戶，其中有多少人買過特定險別

for r in MAIN_ROUTES:
    cust_2020_r = data.query(" year == @STATS_YEAR & iroute==@r & cate=='CA_PS' ")["id"].unique() # 2020年使用過通路_r的客戶
    df = data.query(" year==@STATS_YEAR & id in @cust_2020_r ")
    print(
        df.groupby("cate")["id"].nunique() / df["id"].nunique()
        )

注意!! 以下只處理"車險"

In [8]:
IINS = 'CA_PS'

for STATS_YEAR in range(2017, 2021):

    cust_main = data.query("year==@STATS_YEAR & cate==@IINS ")["id"].unique() #2020有買車險的客戶
    sub_orders = data.query("year<=@STATS_YEAR & id in @cust_main")
    print("=====main group:=====")
    print(f"orders: {len(sub_orders):,.0f} people: {len(cust_main):,.0f}" )
    feat_main = gf.group_summary(sub_orders)

    print("=====by route:=====")
    """
    注意: 此處是{y年}年有用{通路r}買車險的客戶，和續保與否無關
    """
    dictFT_r = {}
    for r in MAIN_ROUTES:
        cust_r = data.query("year==@STATS_YEAR & cate==@IINS & iroute==@r")["id"].unique() #2020有用過通路_r買車險的客戶
        sub_orders = data.query("year<=@STATS_YEAR & id in @cust_r")
        print(f"{r}:")
        print(f"orders: {len(sub_orders):,.0f} people: {len(cust_r):,.0f}" )
        dictFT_r[r] = gf.group_summary(sub_orders)

    print("=====by src & dst:=====")
    """
    注意: 此處是{y年}年有用{通路r}買車險的客戶 且 {y-1年}使用{通路s}買車險的客戶，和續保就有關係了
    """
    dictFT_s_r = {}
    for s in MAIN_ROUTES:
        cust_c_previousYear = data.query("year==@STATS_YEAR-1 & cate==@IINS & iroute==@s")["id"].unique()
        for r in MAIN_ROUTES:
            cust_c_r = data.query("year==@STATS_YEAR & cate==@IINS & iroute==@r & id in @cust_c_previousYear")["id"].unique()
            sub_orders = data.query("year<=@STATS_YEAR & id in @cust_c_r")
            print(f"{s}, {r}:")
            print(f"orders: {len(sub_orders):,.0f} people: {len(cust_c_r):,.0f}" )
            dictFT_s_r[(s, r)] = gf.group_summary(sub_orders)
    summary = {
        "main":feat_main,
        "r":dictFT_r,
        "src_dst":dictFT_s_r
    }
    import pickle
    with open(f"./tem/group_feat_summary_{STATS_YEAR}.pickle", "wb") as f:
        pickle.dump(summary, f)

=====main group:=====
orders: 861,049 people: 678,887
=====by route:=====
CA:
orders: 312,946 people: 241,039


d:\新安\A&H 二周目\Q4\src\group_feat.py:421: RuntimeWarning: invalid value encountered in double_scalars
  clm_amtCust = subset["clmAmt"].sum() / subset_clm["id"].nunique()
d:\新安\A&H 二周目\Q4\src\group_feat.py:426: RuntimeWarning: invalid value encountered in double_scalars
  clm_cnOrder = subset_clm["clmCnt"].sum(
d:\新安\A&H 二周目\Q4\src\group_feat.py:430: RuntimeWarning: invalid value encountered in double_scalars
  clm_amtOrder = subset["clmAmt"].sum() / subset_clm["clmCnt"].sum()


KA:
orders: 234,393 people: 169,823
JB:
orders: 102,147 people: 81,547
BA:
orders: 89,079 people: 69,891
KB:
orders: 90,062 people: 72,524
=====by src & dst:=====
CA, CA:
orders: 0 people: 0
CA, KA:
orders: 0 people: 0
CA, JB:
orders: 0 people: 0
CA, BA:
orders: 0 people: 0
CA, KB:
orders: 0 people: 0
KA, CA:
orders: 0 people: 0
KA, KA:
orders: 0 people: 0
KA, JB:
orders: 0 people: 0
KA, BA:
orders: 0 people: 0
KA, KB:
orders: 0 people: 0
JB, CA:
orders: 0 people: 0
JB, KA:
orders: 0 people: 0
JB, JB:
orders: 0 people: 0
JB, BA:
orders: 0 people: 0
JB, KB:
orders: 0 people: 0
BA, CA:
orders: 0 people: 0
BA, KA:
orders: 0 people: 0
BA, JB:
orders: 0 people: 0
BA, BA:
orders: 0 people: 0
BA, KB:
orders: 0 people: 0
KB, CA:
orders: 0 people: 0
KB, KA:
orders: 0 people: 0
KB, JB:
orders: 0 people: 0
KB, BA:
orders: 0 people: 0
KB, KB:
orders: 0 people: 0
=====main group:=====
orders: 1,511,969 people: 668,386
=====by route:=====
CA:
orders: 497,586 people: 207,978
KA:
orders: 426,553 peopl

檢查

In [26]:
# 檢查續保人數是否"接近" (不會相等，因為沒有看全部的通路)
prev = e.used_data.query(" cate=='CA_PS' & year==2019 ")
this = e.used_data.query(" cate=='CA_PS' & year==2020 ")
print(this.query(" id.isin(@prev['id']) ")["id"].nunique()) # 總續保人數
print(
    sum([d.query("  variable=='n_person' & year==2020 ")["value"].iloc[0] for r, d in dictFT_s_r.items()])
) # 涵蓋人數


528850
518450


In [142]:
with open("./tem/group_feat_summary.pickle", "rb") as f:
    summary = pickle.load(f)

In [145]:
col_orders = ["variable_type", "variable", "cate", "year_type", "value"]
data_main = [
                summary["main"].query(" year==2020 & variable_type.str.match('^(?!.*iroute).*') ")[col_orders]\
                .set_index(col_orders[:-1])\
                .rename(columns={"value": "main"})
            ]
data_r = [
            data.query(" year==2020 & variable_type.str.match('^(?!.*iroute).*') ")[col_orders]\
                .set_index(col_orders[:-1])\
                .rename(columns={"value": r})
            for r, data in summary["r"].items()
        ]

data_s_r = [
            data.query(" year==2020 & variable_type.str.match('^(?!.*iroute).*') ")[col_orders]\
                .set_index(col_orders[:-1])\
                .rename(columns={"value": r})
            for r, data in summary["src_dst"].items()
        ]

data_con_Q1 = data_main + data_r + data_s_r
compare_mat_Q1 = pd.concat(data_con_Q1, axis=1)
# compare_mat_Q1.reset_index(inplace=True)

In [158]:
compare_mat_Q1.loc["contribution"].loc["plyAmt_cust"]

,,main,CA,KA,JB,BA,KB,"(CA, CA)","(CA, KA)","(CA, JB)","(CA, BA)",...,"(BA, CA)","(BA, KA)","(BA, JB)","(BA, BA)","(BA, KB)","(KB, CA)","(KB, KA)","(KB, JB)","(KB, BA)","(KB, KB)"
cate,year_type,,,,,,,,,,,,,,,,,,,,,
CA_PS,this_year,7753.796343,10814.277251,8250.168002,6589.754698,7349.199095,6651.33582,9032.481942,12850.273726,10695.268312,12298.622913,...,16563.840529,10541.474857,8834.869215,7082.266974,10917.512681,15822.299839,11281.704804,8822.967665,10197.619677,6983.161713
F_PS,this_year,2397.926589,2210.46991,2536.930556,2287.547776,2394.906017,2407.886294,2235.449031,2408.694611,2253.371795,2159.942857,...,2004.727273,3143.957447,2502.657143,2430.806065,2300.083333,2030.125,2513.305882,2124.6875,2783.454545,2421.288385
H_GP,this_year,1631.660308,1431.736098,1764.938316,1285.324503,1726.430345,2123.122078,1408.503045,1330.0375,1514.193548,1908.214286,...,2221.416667,1953.0,881.5,1813.955335,2934.666667,1985.083333,2468.568182,2222.333333,2794.615385,2434.611452
H_PS,this_year,3811.395427,4309.604297,4210.276509,3674.871179,3357.541243,3572.692695,4383.540395,4743.652778,3749.745098,3392.666667,...,3500.102564,4560.337838,4029.3,3472.104124,3252.208333,3725.96,3897.559633,3806.166667,3185.173913,3614.941674
H_TV,this_year,500.931698,452.277365,547.319192,540.691721,457.983142,456.43975,419.935882,496.47482,697.307692,507.166667,...,487.538462,614.538462,489.25,491.196341,733.083333,266.384615,523.509434,281.666667,147.4,457.992366


In [86]:
compare_mat_Q1 = compare_mat_Q1.T

In [ ]:

for c, col in compare_mat_Q1.filter(regex=".*Prs").T.iterrows():
    # prs拆解
    tem = np.array(col.fillna('nan,nan,nan').str.split(',').to_list()).astype(float)
    # 新增新欄位
    for idx, pr_name in enumerate(['_Pr25', '_Pr50', '_Pr75']):
        new_col = pd.MultiIndex.from_tuples([[c[0], c[1].replace(r'_Prs', pr_name), *c[2:]]])
        print(compare_mat_Q1.shape, len(tem[:, idx].flatten()))
        compare_mat_Q1[new_col] = tem[:, idx].flatten()

In [116]:
new_col

MultiIndex([('basic_info', 'age_Pr25', nan, 'this_year')],
           )

In [130]:
new_col = [c[0], c[1].replace(r'_Prs', pr_name), c[2], c[3]]
print(new_col)

compare_mat_Q1[new_col]# = np.arange(31).reshape(-1, 1)
# compare_mat_Q1[new_col] = tem[:, idx]
# compare_mat_Q1[new_col] = tem[:, idx].flatten()

['basic_info', 'age_Pr25', nan, 'this_year']


KeyError: "None of [Index(['basic_info', 'age_Pr25', nan, 'this_year'], dtype='object')] are in the [columns]"

In [ ]:
compare_mat_Q1

In [69]:
compare_mat_Q1.shape

(31, 104)

In [57]:
tem[:, 1]

array([16230.,    nan, 81500.,  2400., 65703.,    nan,    nan,    nan,
          nan,    nan,    nan,    nan, 81500.,    nan,    nan,    nan,
          nan,    nan,  2200.,    nan,    nan,    nan,    nan,    nan,
       65703.,    nan,    nan,    nan,    nan,    nan,    nan])

In [9]:
for _, row in compare_mat_Q1[compare_mat_Q1["variable"].str.match(r'.*Prs')].iterrows():
    row["value"]

KeyError: 'value'

In [6]:
with open("./tem/group_feat_summary.pickle", "rb") as f:
    summary = pickle.load(f)
    
col_orders = ["variable_type", "variable", "cate", "year_type", "value"]
data_main = [
                summary["main"].query(" year==2020 & variable_type.str.match('^(?!.*iroute).*') ")[col_orders]\
                .set_index(col_orders[:-1])\
                .rename(columns={"value": "main"})
            ]
data_r = [
            data.query(" year==2020 & variable_type.str.match('^(?!.*iroute).*') ")[col_orders]\
                .set_index(col_orders[:-1])\
                .rename(columns={"value": r})
            for r, data in summary["r"].items()
        ]

data_s_r = [
            data.query(" year==2020 & variable_type.str.match('^(?!.*iroute).*') ")[col_orders]\
                .set_index(col_orders[:-1])\
                .rename(columns={"value": r})
            for r, data in summary["src_dst"].items()
        ]

data_con_Q1 = data_main + data_r + data_s_r
compare_mat_Q1 = pd.concat(data_con_Q1, axis=1)
compare_mat_Q1.reset_index(inplace=True)

In [7]:
plot_data = compare_mat_Q1.query("cate.isna() | cate=='CA_PS'")


雷達圖: 比較不同族群間的特徵
1. 數據
2. 買過的險別(人數%、歷年，radar_groupPlt(year_type="this_year")改成該年)
3. 此年的車種占比(保單%)

In [9]:
from dataclasses import dataclass, field
from typing import List
import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.express as px
from plotly.subplots import make_subplots
from collections import defaultdict


QUERY_BASIC 要再加上Prs的處理 (至少要取pr50)

In [10]:
@dataclass
class ft_query:
    variable_type: List[str] = field(default_factory=list)
    variable: List[str] = field(default_factory=list),
    variable_aliases: List[str] = field(default_factory=list)
    year_type: str = 'this_year'

QUERY_BASIC = ft_query(
    variable_type=[],
    variable=["female_rate", "age_Avg", "marr_pct", "carAge_Avg", "carCnt_Avg", "seqRoutes_Avg", "plyAmt_cust", "clmRate", "clmAmt_cust"],
    variable_aliases=["女性占比", "Avg年齡", "未婚占比", "Avg車齡", "Avg車數", "Avg連續通路使用", "保費/人", "總賠付/總保費", "賠付金/人"],
    year_type=''
)

QUERY_CARTYPE = ft_query(
    variable_type=["car_info"],
    variable=["pct_小客車", "pct_機車", "pct_進口車", "pct_小貨車", "pct_NA", "pct_電動自行車", "pct_其他", "pct_大型重機"],
    variable_aliases=["小客車", "機車", "進口車", "小貨車", "未填", "電動自行車", "其他", "大型重機"],
    year_type="this_year"
)

QUERY_IINSBUY = ft_query(
    variable_type=["iins_type_bought"],
    variable=["CA_PS", "F_PS", "H_GP", "H_PS", "H_TV"],
    variable_aliases=["車險(任意)", "住火", "團傷", "個傷", "旅平"],
    year_type='~this_year'
)

def radar_plot(df, fts: ft_query, base_col:str="main", compare_cols:list=["KA", "BA", "JB", "BA", "CA", "KB"], default_show:list=["KA", "JB"], showlegend=True, data_save_name=''):
    data = df.copy()
    # filter
    if len(vt:=fts.variable_type)>0:
        data = data.query("variable_type.isin(@vt)")
    if len(v:=fts.variable)>0:
        data = data.query("variable.isin(@v)")
    if fts.year_type in ["this_year", "~this_year"]:
        data = data.query("year_type==@fts.year_type")
    # divided into base_ser v.s. comparing_df
    data = data.set_index("variable").loc[fts.variable].reset_index() # 照指定的順序排列
    data = data.set_index(["variable_type", "variable", "cate", "year_type"])
    # 儲存data
    if data_save_name:
        data_ = data.copy()
        data_["ft_name"] = fts.variable_aliases
        data_.to_csv(f'./notebook/特徵群組分析/plotData_{data_save_name}.csv')

    base_ser = data[base_col]
    comparing_df = data[compare_cols]
    # comparing_df/base_ser and standardlized
    df_div = comparing_df / base_ser.values.reshape(-1, 1)
    
    fig = go.Figure()
    # color red for ring=1
    fig.add_trace(
            go.Scatterpolar(
                r=np.ones(len(fts.variable_aliases)),
                theta=fts.variable_aliases,
                meta=base_ser.values,
                name=str(base_col),
                mode="lines+markers",
                line={"width":0, "color":"red"},
                marker={"size":.1},
                fill="toself",
                opacity=.2,
                hovertemplate="%{meta:.2f}",
                legendgroup=str(base_col),
                showlegend=showlegend
            )
        )

    c_idx, colors = 0, px.colors.qualitative.Vivid # 指定顏色
    for name, ser in df_div.T.iterrows():
        # print(ser)
        fig.add_trace(
            go.Scatterpolar(
                r=ser.values,
                theta=fts.variable_aliases,
                meta=data[name],
                name=str(name),
                mode="lines+markers",
                line={"width":1, "color":colors[c_idx]},
                hovertemplate="%{meta:.2f}",
                visible="legendonly" if name not in default_show else None,
                legendgroup=str(name),
                showlegend=showlegend
            )
        )
        c_idx += 1

    return fig

def radar_groupPlt(df, title='', base_col:str="main", compare_cols:list=["KA", "BA", "JB", "BA", "CA", "KB"], default_show:list=["KA", "JB"], data_save_name=''):
    # 基本資訊
    fig = radar_plot(plot_data, QUERY_BASIC, base_col=base_col, compare_cols=compare_cols, default_show=default_show, data_save_name=data_save_name)
    # 買過的險別
    fig_iinsBuy = radar_plot(plot_data, QUERY_IINSBUY, base_col=base_col, compare_cols=compare_cols, default_show=default_show, showlegend=False, data_save_name=data_save_name+"_iinsbuy")
    # 車種
    fig_carType = radar_plot(plot_data, QUERY_CARTYPE, base_col=base_col, compare_cols=compare_cols, default_show=default_show, showlegend=False, data_save_name=data_save_name+"_cartype")

    subplots = make_subplots(
        1, 3,
        subplot_titles= ["", "歷年曾買過的險別(人數%)", "此年投保車種(保單%)"], 
        specs=[[{"type":"polar"}, {"type":"polar"}, {"type":"polar"}]]
    )
    subplots.add_traces(fig.data, 1, 1)
    subplots.add_traces(fig_iinsBuy.data, 1, 2)
    subplots.add_traces(fig_carType.data, 1, 3)
    subplots.update_layout(title=f"<b>2020年 使用各通路族群描述: {title}</b>")
    return subplots

作圖: radar_dict儲存各種對比的雷達圖
1. agg: 不分通路
2. from: 去年從指定通路出發
3. to: 今年抵達指定通路

In [11]:

radar_dict = defaultdict(dict)
# 不看轉移
radar_dict["agg"] = radar_groupPlt(plot_data, base_col="main", compare_cols=["KA", "KB", "CA", "JB", "BA"], default_show=["KA", "JB"], data_save_name="main")
# 看轉移
for r in ["KA", "KB", "CA", "BA", "BA", "JB"]:
    from_r = [c for c in plot_data.columns if isinstance(c, tuple) and c[0]==r]
    radar_dict["from"][r] = radar_groupPlt(plot_data, title=f"src from {r}", base_col="main", compare_cols=from_r, default_show=[(r, r), (r, 'JB')], data_save_name=f'from_{r}')
    to_r = [c for c in plot_data.columns if isinstance(c, tuple) and c[1]==r]
    radar_dict["to"][r] = radar_groupPlt(plot_data, title=f"dst to {r}", base_col="main", compare_cols=to_r, default_show=[('KA', r), ('JB', r)], data_save_name=f'to_{r}')
    
    

In [12]:

with open(f"./markdown/群組特徵分析/agg.txt", "w") as f:
    f.write(
        pyo.plot(radar_dict["agg"], auto_open=False, output_type="div", include_plotlyjs=False)
    )

In [13]:
for k, dict_lv1 in radar_dict.items():
    if k == 'agg':
        pyo.plot(radar_dict[k], filename=f"./markdown/群組特徵分析/agg.html", auto_open=False)
        with open(f"./markdown/群組特徵分析/agg.txt", "w") as f:
            f.write(
                pyo.plot(radar_dict[k], auto_open=False, output_type="div", include_plotlyjs=False)
            )
        continue
    for r, fig in dict_lv1.items():
        with open(f"./markdown/群組特徵分析/{k}_{r}.txt", "w") as f:
            pyo.plot(fig, filename=f"./markdown/群組特徵分析/{k}_{r}.html")
            f.write(
                    pyo.plot(fig, auto_open=False, output_type="div", include_plotlyjs=False)
                )

radar_dict["from"]["KA"]
radar_dict["to"]["JB"]
radar_dict["agg"]

with open('./notebook/特徵群組分析/radar_dict.pickle', 'wb') as f:
    pickle.dump(radar_dict, f)


In [34]:
radar_dict["agg"]

radar_dict 整理成一個html

In [42]:
# %% 異常探勘?
# KAKA H_GS
# KAJB H_PS

# (dictOR_src_dst[("KA", "JB")]["clmAmt"]>0).sum()
# dictOR_src_dst[("KA", "JB")].groupby("cate").agg(
#     dict(id = "nunique",
#     clmAmt = "sum")
# )
dictFT_s_r[("KA", "KA")].query("variable_type=='iins_type_bought'")
# dictOR_src_dst[("KA", "KA")].query("cate=='CA_PS'")

    

,year,variable,value,variable_type,year_type,cate
0,2020.0,CA_PS,0.974754,iins_type_bought,this_year,NaN
1,2020.0,F_PS,0.03167,iins_type_bought,this_year,NaN
2,2020.0,H_GP,0.018515,iins_type_bought,this_year,NaN
3,2020.0,H_PS,0.032658,iins_type_bought,this_year,NaN
4,2020.0,H_TV,0.017497,iins_type_bought,this_year,NaN
0,2020.0,CA_PS,0.974754,iins_type_bought,~this_year,NaN
1,2020.0,F_PS,0.03167,iins_type_bought,~this_year,NaN
2,2020.0,H_GP,0.018515,iins_type_bought,~this_year,NaN
3,2020.0,H_PS,0.032658,iins_type_bought,~this_year,NaN
4,2020.0,H_TV,0.017497,iins_type_bought,~this_year,NaN


開發中:
還少一個客齡 (~統計年)

In [11]:
import tqdm, datetime


In [19]:

used_data = e.raw_data.query("""
    sex.str.len() <= 1 & \
    plyAmt >= 0 & \
    date >= '2017-01-01' & \
    date <= '2022-03-31' & \
    cate != 'H_CV'
    """).copy()

used_data.loc[used_data["cate"].str.startswith("C"), "cate"] = "Car"

data_holder = []
for i in tqdm.tqdm(used_data["cate"].unique()):
    print(f"cate={i}")
    for y in range(2017, 2022):
        print(f"year={y}")
        s = datetime.datetime.today()
        thisYear = used_data.query("""
                    year==@y & \
                    cate==@i
                    """)["id"].astype(str)
        prevYear = used_data.query("""
                        year==@y-1 & \
                        cate==@i
                        """)["id"].astype(str)
        long_time_ago = used_data.query("""
                    year<@y-1 & \
                    cate==@i
                    """)["id"].astype(str)
        new = thisYear[~thisYear.isin(prevYear) & ~thisYear.isin(long_time_ago)].nunique()
        trans = thisYear[thisYear.isin(prevYear)].nunique() #  & ~thisYear.isin(long_time_ago)
        prev_old = thisYear[~thisYear.isin(prevYear) & thisYear.isin(long_time_ago)].nunique()
        sink = prevYear[~prevYear.isin(thisYear)].nunique()

        data_holder.append(
            pd.Series([i, y, new, trans, prev_old, sink], index=["cate", "year", "新", "續", "回", "流失"])
            )

data_holder = pd.DataFrame(data_holder)


In [44]:
tem  =e.used_data.query("cate.isin(['Cf_PS', 'CA_PS'])")

In [58]:
t_2020 = e.raw_data.query("cate.isin(['CA_PS']) & year==2020")
t_2019 = e.raw_data.query("cate.isin(['CA_PS']) & year==2019")

In [59]:
t_2020["id"].nunique()

751395

In [60]:
t_2020[t_2020["id"].isin(t_2019["id"])]["id"].nunique()

528850

In [46]:
len(set(t_2020['id'].unique()).intersection(t_2019['id'].unique()))

528850

In [71]:
y, i = 2020, "CA_PS"

thisYear = e.used_data.query("""
            year==@y & \
            cate==@i
            """)["id"].astype(str)
prevYear = e.used_data.query("""
                year==@y-1 & \
                cate==@i
                """)["id"].astype(str)
long_time_ago = e.used_data.query("""
            year<@y-1 & \
            cate==@i
            """)["id"].astype(str)
new = thisYear[~thisYear.isin(prevYear) & ~thisYear.isin(long_time_ago)].nunique()
trans = thisYear[thisYear.isin(prevYear)]

In [73]:
trans.nunique()

528850

***
圖表說明:
1. legend表示 **族群**:
    - **main**: 2020年所有買過車險的客戶
    - **KA**: 2020年所有使用KA買過車險的客戶
    - 以此類推
2. 雷達圖:
    - **散布位置**: 族群的第i個特徵(例: 未婚占比) / **main**的相同特徵。 例: **未婚占比_JB / 未婚占比_main** = 0.29/0.47 = **0.61**
    <br>(i.e. 把特徵分數標準化，讓所有特徵有相同的尺度)
    - <span style="color:#F6A198">**紅色範圍**</span>: 半徑為1的範圍，代表**main**，即比較的基礎。
***

### by通路(不分來源):

例: 2020年使用 {KA} 買過車險的客戶。

1. JB相較其他族群:
    - 有較低的**未婚占比**: 29% v.s. 47%(main)
    - **保費/人**最低: 6,589 v.s. 7,753(main), 10,814(CA)
    - 有較低的**賠付金/人**: 48,927 v.s. 54,696(main)
    - 但有偏高的**
    - 投保**車種**以小客車(53%)、機車(34%)為主

2. 明顯特徵:
    - **未婚占比**: KB(64%)、BA(61%)明顯較高；CA(34%)、JB(29%)則明顯偏低。 (v.s. main, 47%)
    - **Avg車齡**: 大致落在8~9年，以KA(9.78年)最高、CA(5.93年)年最低。
    - **Avg車數**: 大致落在1.3台左右，以KA(1.43台)最高、CA(1.18台)最低
    - **Avg通路使用**: 大多位於3次以下，為CA(3.58次)、KA(3.47次)超過3次。值得注意的是，JB(2.9次)是黏著度第三名。
    - 

In [21]:
data_holder

,cate,year,新,續,回,流失
0,Car,2017,789053,0,0,0
1,Car,2018,224514,553300,0,235753
2,Car,2019,205438,103949,50826,216601
3,Car,2020,222176,108941,57659,218568
4,Car,2021,198136,117135,67135,230420
5,F_PS,2017,42224,0,0,0
6,F_PS,2018,13504,34157,0,8067
7,F_PS,2019,39243,9753,2059,7797
8,F_PS,2020,34803,34560,3572,9128
9,F_PS,2021,20340,30767,1583,12401


In [22]:
t = used_data.groupby("cate")["id"].nunique()
t = pd.concat([
    data_holder,
    pd.DataFrame(t).assign(year="不分年").reset_index().rename(columns={"id":"總人數"})
    ])
t.sort_values(["cate", "year"], inplace=True)
# t.to_csv("./data/人數_byYear_Cate.csv", index=False)


In [23]:
t

,cate,year,新,續,回,流失,總人數
0,Car,2017,789053.0,0.0,0.0,0.0,NaN
1,Car,2018,224514.0,553300.0,0.0,235753.0,NaN
2,Car,2019,205438.0,103949.0,50826.0,216601.0,NaN
3,Car,2020,222176.0,108941.0,57659.0,218568.0,NaN
4,Car,2021,198136.0,117135.0,67135.0,230420.0,NaN
0,Car,不分年,NaN,NaN,NaN,NaN,1684454.0
5,F_PS,2017,42224.0,0.0,0.0,0.0,NaN
6,F_PS,2018,13504.0,34157.0,0.0,8067.0,NaN
7,F_PS,2019,39243.0,9753.0,2059.0,7797.0,NaN
8,F_PS,2020,34803.0,34560.0,3572.0,9128.0,NaN
